In [1]:
# use docker to parse grib file
#docker pull osgeo/gdal
#docker run --rm -v "C:/Users/WeilunShi/OneDrive - CGIAR/Desktop/Food Crisis and Insecurity/0.external_data/ERA5_data:/data" osgeo/gdal:latest gdal_translate -of GTiff /data/ERA_2.grib /data/output_2.tif

import numpy as np
import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape

# Read the .tif file with Rasterio
with rasterio.open(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\ERA5_evaporation\output_2.tif') as src:
    image = src.read(1)  # Read the first band
    transform = src.transform

# Convert the image data type to int16
image = image.astype(np.int16)
# Extract shapes from the raster data
shapes_and_values = list(shapes(image, transform=transform))
geometries = [shape(s[0]) for s in shapes_and_values]
values = [s[1] for s in shapes_and_values]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': geometries, 'value': values})

# Set the Coordinate Reference System (CRS) to match the input raster
gdf.crs = src.crs


# set CRS to 4236
gdf = gdf.to_crs(epsg=4326)



In [2]:
gdf.describe()

,value
count,1921.000000
mean,3773.698594
std,4848.159114
min,0.000000
25%,0.000000
50%,0.000000
75%,9999.000000
max,9999.000000


In [ ]:


# read geojson
areas = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\intermediate\output9.geojson')


# spatial join
result = gpd.sjoin(areas, gdf, how='left', op='intersects')

# rename
result = result.rename(columns={'value': 'total_evaporation'})

#parse missing value
result.loc[result['total_evaporation']==9999] = np.nan

# summarize
results = result.groupby(['date','title'])['total_evaporation'].mean().reset_index()

# merge
results = results.merge(result, on=['title','date'], how='right')



results.drop(columns=['total_evaporation_y','index_right'], inplace=True)
#rename
results.rename(columns={'total_evaporation_x':'total_evaporation'}, inplace=True)


#drop duplicates
results.drop_duplicates(inplace=True)

# convert dataframe to geodataframe
results = gpd.GeoDataFrame(results, geometry='geometry')

# save as geojson
results.to_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\intermediate\output10.geojson', driver='GeoJSON')